In [2]:
import pandas as pd
import numpy as np

In [3]:
rewards_tab = pd.read_csv("gridworld.csv")

states = rewards_tab['s'].unique()

empty_array = np.array([0. for i in range(len(states))])

def which_state_num(states, state):
    return np.where(states == state)[0][0]
    
rewards_tab["s_num"] = rewards_tab['s'].apply(lambda state: which_state_num(states, state)) 
rewards_tab["s'_num"] = rewards_tab["s'"].apply(lambda state: which_state_num(states, state)) 

rewards_tab

,s,a,s',r,s_num,s'_num
0,"(1,1)",N,"(1,1)",-1,0,0
1,"(1,1)",E,"(1,2)",0,0,1
2,"(1,1)",S,"(2,1)",0,0,5
3,"(1,1)",W,"(1,1)",-1,0,0
4,"(1,2)",N,"(5,2)",10,1,21
5,"(1,2)",E,"(5,2)",10,1,21
6,"(1,2)",S,"(5,2)",10,1,21
7,"(1,2)",W,"(5,2)",10,1,21
8,"(1,3)",N,"(1,3)",-1,2,2
9,"(1,3)",E,"(1,4)",0,2,3


## Solve Bellman Equation

This approach uses the method as discussed in Chapter 3: Finite Markov Decision Processes

Because it is a system of linear equations, I fill in the values as a matrix, switching the standard belman equation to have v(s) on the right side and the constant - rewards - on the left side so that we can solve using linear algebra.

In [4]:
arr2d_list = []
rew_list = []

for state in states:
    state_num = which_state_num(states, state)

    state_df = rewards_tab.loc[rewards_tab['s'] == state]
    
    array_state = empty_array.copy()

    # all the values of future possible states
    prime_df = state_df["s'_num"].value_counts().reset_index()

    # for each future possible state, fill out 
    for row in range(prime_df.shape[0]):
        s_num = prime_df.loc[row]['index']
        count = prime_df.loc[row]["s'_num"]

        array_state[s_num] = .25*.9*count

    array_state[state_num] = array_state[state_num] - 1
    
    tot_rew = - state_df['r'].sum()*.25

    arr2d_list.append(array_state)
    
    rew_list.append(tot_rew)

arr2d = np.array(arr2d_list)

rew = np.array(rew_list)


In [5]:
values_vec = np.linalg.solve(arr2d,rew)

values_vec.reshape(5,5)

array([[ 3.30899634,  8.78929186,  4.42761918,  5.32236759,  1.49217876],
       [ 1.52158807,  2.99231786,  2.25013995,  1.9075717 ,  0.54740271],
       [ 0.05082249,  0.73817059,  0.67311326,  0.35818621, -0.40314114],
       [-0.9735923 , -0.43549543, -0.35488227, -0.58560509, -1.18307508],
       [-1.85770055, -1.34523126, -1.22926726, -1.42291815, -1.97517905]])

#### Bellman Optimality Equation

The above values are based on the specific policy of equal probability of any of the four directions. If we look at the Bellman Optimality Equation solution, we will see the values under optimal policy. 

Actually I'm realizing this is a difficult system of equations to solve.

E.g.

$v*(0) = max_a [ (-1 + .9 v*(0)),  (-1 + .9 v*(0)),  (0 + .9 v*(1)),  (0 + .9 v*(5)) ]$
$v*(1) = max_a [ (10 + .9 v*(21)),  (10 + .9 v*(21)),  (10 + .9 v*(21)),  (10 + .9 v*(21)) ]$
   
  so $v*(1) = 10 + .9*v(21)$

If $v*(21) = 0$ then the optimal value of $v*(1)$ is its reward for going in any direction from it, 10. 


In [191]:
for s_num in rewards_tab.s_num.unique():
    
    print(s_num)

    state_df = rewards_tab.loc[rewards_tab['s_num'] == s_num].reset_index()

    all_vals = []

    for row in range(state_df.shape[0]):
        sprime_num = state_df.loc[row]["s'_num"]
        sprime_rew = state_df.loc[row]['r']

        sprime_val = value_df.loc[sprime_num, "value"]

        val = sprime_rew + .9*sprime_val
    
        all_vals.append(val)

    best_action = state_df["a"].values[np.where(all_vals == max(all_vals))[0][0]]
        
    val_best_action = np.array(all_vals)[np.where(all_vals == max(all_vals))]

    print(val_best_action)
        
    
    

0
[9.]
1
[10. 10. 10. 10.]
2
[9.]
3
[5. 5. 5. 5.]
4
[4.5]
5
[0. 0. 0.]
6
[9.]
7
[0. 0. 0. 0.]
8
[4.5]
9
[0. 0. 0.]
10
[0. 0. 0.]
11
[0. 0. 0. 0.]
12
[0. 0. 0. 0.]
13
[0. 0. 0. 0.]
14
[0. 0. 0.]
15
[0. 0. 0.]
16
[0. 0. 0. 0.]
17
[0. 0. 0. 0.]
18
[0. 0. 0. 0.]
19
[0. 0. 0.]
20
[0. 0.]
21
[0. 0. 0.]
22
[0. 0. 0.]
23
[0. 0. 0.]
24
[0. 0.]


## Dynamic Programming Approaches

These approaches uses dynamic programming by iteratively solving the Bellman equation as discussed in Chapter 4: Dynamic Programming

### Policy Evalulation

In [94]:
threshold = .005

i = 0

value_df = pd.DataFrame({"s":rewards_tab['s'].unique()})
value_df['s_num'] = value_df['s'].apply(lambda x: which_state_num(states, x))
value_df['value']  = 0

cont = True

while(cont):
    
    delta = 0
    
    for s_num in rewards_tab['s_num'].unique():    

        v_old = value_df.loc[s_num, "value"]

        state_df = rewards_tab.loc[rewards_tab['s_num'] == s_num].reset_index()

        v_sum = 0
        
        for row in range(state_df.shape[0]):
            sprime_num = state_df.loc[row]["s'_num"]
            sprime_rew = state_df.loc[row]['r']

            sprime_val = value_df.loc[sprime_num, "value"]

            v_sum = v_sum + (sprime_rew + .9*sprime_val)

        delta = max(delta, abs(v_sum - v_old))
            
        value_df.loc[s_num, 'value'] = v_sum
        
    cont = delta > threshold
    
    i += 1
    
print("Delta less than threshold after {} iterations".format(i))

value_df['value'].values.reshape(5,5)


Delta less than threshold after 21 iterations


array([[ 3.33187441,  8.80774354,  4.44554289,  5.33882158,  1.50938862],
       [ 1.54268712,  3.01045441,  2.26685283,  1.92338122,  0.56309835],
       [ 0.07104725,  0.75578829,  0.68919979,  0.37342236, -0.38821464],
       [-0.95380217, -0.41821709, -0.33913689, -0.57071105, -1.16853777],
       [-1.83808956, -1.32810669, -1.21367081, -1.40817498, -1.96080363]])

### Policy Improvement

Adding in policy improvement in each iteration. To simplify things, not using stoachstic policies. Going to initialize with policy "always go north" for all states.

In [195]:

# note this function differs in a couple ways from the policy evaluation done above
def policy_evaluation(rewards_tab, value_df, policy_df, threshold):
        
    i = 0

    cont = True

    while(cont):

        delta = 0

        for s_num in rewards_tab['s_num'].unique():    

            v_old = value_df.loc[s_num, "value"]

            policy_action = policy_df.loc[s_num, "pi"]

            sprime_num = rewards_tab.loc[(rewards_tab['s_num'] == s_num) & (rewards_tab['a'] == policy_action)]["s'_num"].values[0]
            sprime_rew = rewards_tab.loc[(rewards_tab['s_num'] == s_num) & (rewards_tab['a'] == policy_action)]["r"].values[0]

            sprime_val = value_df.loc[value_df['s_num'] == new_state]['value'].values[0]

            v_new = sprime_rew + .5*sprime_val

            delta = max(delta, abs(v_new - v_old))

            value_df.loc[s_num, 'value'] = v_new

        cont = delta > threshold

        i += 1
        
    print("{} iterations".format(i))
        
    return value_df

def policy_improvement(rewards_tab, value_df, policy_df):

    policy_stable = True

    for s_num in rewards_tab['s_num'].unique():   

        old_action = policy_df.loc[s_num]["pi"]

        state_df = rewards_tab.loc[rewards_tab['s_num'] == s_num].reset_index()

        all_vals = []

        for row in range(state_df.shape[0]):
            sprime_num = state_df.loc[row]["s'_num"]
            sprime_rew = state_df.loc[row]['r']

            sprime_val = value_df.loc[sprime_num, "value"]

            val = sprime_rew + .5*sprime_val

            all_vals.append(val)

        best_action = state_df["a"].values[np.where(all_vals == max(all_vals))[0][0]]

        if len(best_action) > 1:
            print("more than one best action")

        policy_df.loc[s_num, "pi"] = best_action

        if best_action != old_action:
            policy_stable = False
            
    return policy_stable, policy_df


In [196]:
threshold = 0.005

## value initialization
value_df = pd.DataFrame({"s":rewards_tab['s'].unique()})
value_df['s_num'] = value_df['s'].apply(lambda x: which_state_num(states, x))
value_df['value']  = 0

## being with policy of always going North
policy_df = pd.DataFrame({"pi": ["S" for i in range(25)]})

policy_stable = False

while(policy_stable is False):
    
    value_df = policy_evaluation(rewards_tab, value_df, policy_df, threshold)
    
    policy_stable, policy_df = policy_improvement(rewards_tab, value_df, policy_df)
        
policy_df

2 iterations
2 iterations


,pi
0,E
1,N
2,W
3,N
4,W
5,N
6,N
7,N
8,N
9,N


In [194]:
value_df

,s,s_num,value
0,"(1,1)",0,0.0
1,"(1,2)",1,10.0
2,"(1,3)",2,0.0
3,"(1,4)",3,5.0
4,"(1,5)",4,0.0
5,"(2,1)",5,0.0
6,"(2,2)",6,0.0
7,"(2,3)",7,0.0
8,"(2,4)",8,0.0
9,"(2,5)",9,0.0
